## Large Policy v2: Model-based approach

Given unknown `(s, a, r, sp)` data, find optimal policy. Not all `(s, a)` pairs will be seen in data, so interpolate from neighbors.
- States: |S| = 302020
- Actions: 9 actions
- Discount factor = 0.95

Lisa Fung

Last Updated: 11/9/24

### Data Exploration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
large_data = pd.read_csv("./data/large.csv")
n_states = 302020
n_actions = 9
# n_limit_actions = 5

Large Data Observations

Rewards
- Only 7 unique values: [-10  -5   0   5  10  50 100]
- r=100 only at states sp = 301013, 301111, via actions [1,4]
- sp = 301013
    - s=301012, a=1 (delta_s = +1)
    - s=301014, a=2 (delta_s = -1)
    - s=301113, a=3 (delta_s = -100)
    - s=300413, a=4 (delta_s = +600)
- sp = 301111
    - s=301110, a=1 (delta_s = +1)
    - s=301112, a=2 (delta_s = -1)
    - s=301211, a=3 (delta_s = -100)
    - s=301011, a=4 (delta_s = +100)


Actions
- a = [1,4] are probabilistic
- a = [5,9] are usually 0, occasionally random
    ```

### Approach

- Transition Model: $T(s' - s \mid a)$
- Rewards: $R(s, s')$
- Only take actions $a = 1,2,3,4,5$


#### Transition Model

$T(a, \Delta s) = T(\Delta s \mid a)$
- $|\Delta s| = 9$, $|A| = 5$

In [18]:
large_data['delta_s'] = large_data['sp'] - large_data['s']  # delta_s = sp - s

# Only keep delta_s for actions [1, 4] and 0
# array([-600, -100,   -6,   -1,    0,    1,    6,  100,  600])
delta_s_list = np.sort(large_data[large_data['a'] == 1]['delta_s'].unique())

In [49]:
n_delta_s = 9
n_limit_actions = 5

T = np.zeros((n_limit_actions+1, n_delta_s))
T.shape

(6, 9)

In [29]:
large_data_a_delta_s = large_data[large_data['delta_s'].isin(delta_s_list)][['a', 'delta_s']].value_counts().sort_index(level=[0,1])

In [56]:
# Aggregate transition counts
for (a, delta_s), count in large_data_a_delta_s.items():
    # row: ((a, delta_s), count)
    if a <= n_limit_actions:
        delta_s_idx = np.where(delta_s_list == delta_s)[0][0]
        T[a, delta_s_idx] = count

In [58]:
# Normalize along next state (sp) dimension to divide by N(s, a)
T /= np.sum(T, axis=1, keepdims=True)
T = np.nan_to_num(T, nan=0.0)   # convert nan to 0.0

C:\Users\lisaf\AppData\Local\Temp\ipykernel_20080\2452209420.py:2: RuntimeWarning: invalid value encountered in divide
  T /= np.sum(T, axis=1, keepdims=True)


Reward Model
- $R(s, s')$ uniquely determines reward value

In [ ]:
rewards_list = np.sort(large_data['r'].unique())
rewards_list

array([-10,  -5,   0,   5,  10,  50, 100])

In [73]:
# for r in rewards_list[0:1]:
#     print(f"Reward r = {r}:")
#     print(large_data[large_data['r'] == r][['s', 'sp', 'a', 'r']].value_counts().sort_index())
#     print(large_data[large_data['sp'] == 301112][['s', 'sp']].value_counts())

In [122]:
# Determine if any (s, sp) pair has different rewards
# Answer: no. Reward is uniquely determined by R(s, sp)
large_data_rewards = large_data.groupby(['s', 'sp'])
large_data_rewards['r'].nunique().unique()

array([1])

In [123]:
states_list = np.sort(large_data['s'].unique())
n_seen_states = len(states_list)    # 500

# R = np.zeros((n_seen_states, n_seen_states))
R = large_data.pivot_table(index='s', columns='sp', values='r', aggfunc='first').to_numpy()

In [124]:
# large_data.pivot_table(index='sp', columns='s', values='r', aggfunc='first')[300413][301013]
# # large_data[(large_data['s'] == 300413) & (large_data['sp'] == 301013)]

# Check R indices
test_s = 301014
test_s_idx = np.where(states_list == test_s)[0][0]
test_sp = 301013
test_sp_idx = np.where(states_list == test_sp)[0][0]
R[test_s_idx, test_sp_idx]

np.float64(100.0)

In [128]:
actions_list = np.arange(1, n_limit_actions+1)
actions_list

array([1, 2, 3, 4, 5])

#### Value Iteration


In [ ]:
# finish Sunday 11/10!

def value_iteration(U, n_iters=100, discount=0.95, threshold=1e-3):
    # Update U with intermediate state updates instead of one iteration at a time
    for i in range(n_iters):
        residual = 0    # Maximum change in value among all U[s]
        for s in range(len(states_list)):
            max_Us = 0
            for a in actions_list:
                later_rewards = sum([T[a, delta_s] * U[sp] for delta_s_idx in range(1, n_states+1)])
                # sp = s + delta_s
                max_Us = max(max_Us, R[s] + discount * later_rewards)
            residual = max(residual, abs(U[s] - max_Us))
            U[s] = max_Us
        if residual < threshold:
            print(f"Value iteration converged within threshold {threshold} at iteration {i}\n")
            break
    return U

### Extract Policy from Q Function

In [125]:
# Extract optimal policy pi(s) = a from action-value function Q(s, a)

def extract_policy(Q, mode='random'):
    """
    Limit only to actions 1-4.
    """
    policy = np.zeros(n_states+1)
    # predicable_action = np.random.randint(1, n_actions+1)
    predicable_action = 4
    for s in range(1, n_states+1):
        policy[s] = np.argmax(Q[s, :])
        if policy[s] not in [1, 2, 3, 4]: # Actions [5,9] are usually 0, random
            if mode == 'random':
                policy[s] = np.random.randint(1, 5)
            if mode == 'previous':
                policy[s] = predicable_action
        else:
            predicable_action = policy[s]

    return policy

In [126]:
optimal_policy = extract_policy(Q, mode='random')

NameError: name 'Q' is not defined

In [ ]:
np.unique(optimal_policy, return_counts=True)

NameError: name 'optimal_policy_sarsa' is not defined

In [ ]:
# Write optimal policy to file
with open("large.policy", "w") as file:
    for a in optimal_policy[1:]:
        file.write(f"{int(a)}\n")

### Future Improvements

- Average values of Q(s, a) with some distance-dependent discount for unvisited states s